In [ ]:
import numpy as np 
import pandas as pd
import os
import librosa
import librosa.display

import IPython.display as ipd

import matplotlib.pyplot as plt
print(os.listdir("../input"))

import torchaudio
from sklearn.decomposition import PCA

import seaborn as sns

import time
import gc
from sklearn.metrics import label_ranking_average_precision_score
from gensim.models import Word2Vec


In [ ]:
Labels = pd.read_csv("../input/train_curated.csv")
WavPath = "../input/train_curated/"
Fils = os.listdir(WavPath)
sound, sample_rate = torchaudio.load(WavPath+Fils[7])
ipd.Audio(data=sound[0,:],rate=sample_rate) # load a local WAV file

In [ ]:
x, sr = librosa.load(WavPath+Fils[7])
x.shape

In [ ]:
def get_audio_embedding(fname,n_components=10):
    x, sr = librosa.load(fname)
    X = librosa.stft(x)
    Xdb = librosa.amplitude_to_db(abs(X))
    pca = PCA(n_components=n_components)
    pca.fit(Xdb.T)
    return pca.components_.T

In [ ]:
def get_Xdb(fname):
    x, sr = librosa.load(fname)
    X = librosa.stft(x)
    Xdb = librosa.amplitude_to_db(abs(X))
    return Xdb

In [ ]:
def calculate_overall_lwlrap_sklearn(truth, scores):
    """Calculate the overall lwlrap using sklearn.metrics.lrap."""
    # sklearn doesn't correctly apply weighting to samples with no labels, so just skip them.
    sample_weight = np.sum(truth > 0, axis=1)
    nonzero_weight_sample_indices = np.flatnonzero(sample_weight > 0)
    overall_lwlrap = label_ranking_average_precision_score(
        truth[nonzero_weight_sample_indices, :] > 0, 
        scores[nonzero_weight_sample_indices, :], 
        sample_weight=sample_weight[nonzero_weight_sample_indices])
    return overall_lwlrap

In [ ]:
n_classes = 80

def split_and_label(rows_labels, n_classes):
    '''
    Retrieves a list of all the relevant classes. This is necessary due to 
    the multi-labeling of the initial csv file.
    '''
    row_labels_list = []
    for row in rows_labels:
        row_labels = row.split(',')
        labels_array = np.zeros((n_classes))
        for label in row_labels:
            index = label_mapping[label]
            labels_array[index] = 1
        row_labels_list.append(labels_array)
    return row_labels_list

In [ ]:
# Load in data
df = pd.read_csv('../input/train_curated.csv')
test_df = pd.read_csv('../input/sample_submission.csv')

# Retrieve labels
label_columns = test_df.columns[1:]
label_mapping = dict((label, index) for index, label in enumerate(label_columns))
for col in label_columns:
    df[col] = 0  
df[label_columns] = split_and_label(df['labels'], n_classes)
df['num_labels'] = df[label_columns].sum(axis=1)
print(df.head())

print(test_df.head())

In [ ]:
df.describe()

In [ ]:
start = time.time()
get_audio_embedding(WavPath + Fils[0]).shape
print(time.time() - start)

In [ ]:
#start = time.time()
#embeddings = {str(i):get_audio_embedding(WavPath + fname) for (i,fname) in zip(range(len(Fils)), Fils)}
#print(time.time() - start)
#gc.collect()

In [ ]:
# basic schema nutshell..

x = pd.DataFrame(index=range(1025))
start = time.time()
for key in range(len(Fils)):
    x = x.merge(pd.DataFrame(get_audio_embedding(WavPath + Fils[key]),columns=[Fils[key]+'.'+ str(col) for col in range(10)])
                ,left_index=True,right_index=True)
    #x = x.merge(pd.DataFrame(embeddings[str(key)],columns=[Fils[key]+'.'+ str(col) for col in range(10)])
     #           ,left_index=True,right_index=True)
print(time.time() - start)
gc.collect()

y = pd.DataFrame(get_Xdb(WavPath + Fils[10]))
sns.heatmap(y)
print(x.shape)
print(y.shape)

print(x.corrwith(y[0]))
print(np.argmax(x.corrwith(y[0])))
def convert_Xdb2token(Xdb, hv):
    # for every column Xdb, return hv column name with highest correlation
    cols = hv.columns
    p,n = Xdb.shape
    
    def get_idx(x,hv):
        return np.argmax(hv.corrwith(x))
    
    Xdb = pd.DataFrame(Xdb)
    
    final_array = [ get_idx(Xdb[col],hv) for col in Xdb.columns]
    
    return final_array

yseq = convert_Xdb2token(y, x)
print(yseq)


In [ ]:
start = time.time()
yseq = [convert_Xdb2token(pd.DataFrame(get_Xdb(WavPath+Fils[i])),x) for i in range(len(Fils))]
print(time.time() - start)

In [ ]:
#start = time.time()
#model = Word2Vec(yseq, size=300, window=5, min_count=1, workers=4)
#print(time.time() - start)

In [ ]:
# develop an embedding relating spectra co-occurence..

# need a lexicon defined in spectral domain..

# find all Hz sequences

# Discover a finite vocabulary

# Tokenize S relative to vocabulary

# Discover embedding given token-to-token co-occurence

# apply keras with embedding layer